In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, 
                                                  fixed_length_right=40, 
                                                  remove_stop_words=True)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7221.86it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3820.58it/s]
Building Vocabulary from a datapack.: 100%|██████████| 234263/234263 [00:00<00:00, 1619906.09it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8373.09it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4262.79it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 114424.01it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 122/122 [00:00<00:00, 8167.81it/s]
Processing text_right with chain_transfor

In [8]:
model = mz.models.MatchPyramid()

model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['num_blocks'] = 2
model.params['kernel_count'] = [16, 32]
model.params['kernel_size'] = [[3, 3], [3, 3]]
model.params['dpool_size'] = [3, 10]
model.params['optimizer'] = 'adam'
model.params['dropout_rate'] = 0.1
model.guess_and_fill_missing_params()

model.build()
model.compile()

In [9]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1654600     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
matching_layer_2 (MatchingLayer (None, 10, 40, 1)    0           embedding[0][0]                  
          

In [6]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [10]:
model.load_embedding_matrix(embedding_matrix)

In [18]:
dpool_callback = mz.data_generator.callbacks.DynamicPooling(
    fixed_length_left=10, 
    fixed_length_right=40
)
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20,
    callbacks=[dpool_callback]
)
print('num batches:', len(train_generator))

num batches: 816


In [19]:
test_generator = mz.DataGenerator(
    test_pack_processed,
    batch_size=20,
    callbacks=[dpool_callback]
)
len(test_generator)

118

In [21]:
test_x, test_y = test_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=test_x, y=test_y, batch_size=len(test_y))

In [22]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/20
816/816 [==============================] - 17s 21ms/step - loss: 0.2310
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.509005582393753 - normalized_discounted_cumulative_gain@5(0.0): 0.5851168326689696 - mean_average_precision(0.0): 0.5359601444858333
Epoch 2/20
816/816 [==============================] - 40s 49ms/step - loss: 0.0107
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5194788618507995 - normalized_discounted_cumulative_gain@5(0.0): 0.5908798870843132 - mean_average_precision(0.0): 0.5427507384785866
Epoch 3/20
816/816 [==============================] - 41s 50ms/step - loss: 0.0048
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5370681656118138 - normalized_discounted_cumulative_gain@5(0.0): 0.6042993634090988 - mean_average_precision(0.0): 0.5618086563704341
Epoch 4/20
816/816 [==============================] - 41s 50ms/step - loss: 0.0037
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5375455305428062 - normaliz

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
model.evaluate(test_x, test_y, batch_size=5)

{normalized_discounted_cumulative_gain@3(0.0): 0.5337704847339166,
 normalized_discounted_cumulative_gain@5(0.0): 0.6016249668554398,
 mean_average_precision(0.0): 0.5508765369261149}